In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

In [2]:
lih = HamiltonianSmall('LiH', 1.5)
beh2 = HamiltonianSmall('BeH2', 1.3)

h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

water_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')
water_parity = Hamiltonian('H2O_STO3g_14qubits', 'parity')
water_bk = Hamiltonian('H2O_STO3g_14qubits', 'bk')

ammonia_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')
ammonia_parity = Hamiltonian('NH3_STO3g_16qubits', 'parity')
ammonia_bk = Hamiltonian('NH3_STO3g_16qubits', 'bk')

Consider the set of influential pairs:
\begin{align}
    \mathcal{I}_\mathrm{comp}
    =
    \left\{\left.
        (\Qarrow,\Rarrow)
        \,\right|\,
        \textrm{for all $i$, either $Q_i=R_i$, or $\{Q_i,R_i\}=\{I,Z\}$}
    \right\}
\end{align}

Then the cost function to optimise will be:
\begin{align}
    \mathrm{cost}(\{\beta_i\}_{i=1}^n)
    =
    \sum_{\Qarrow,\Rarrow\in\mathcal{I}_\mathrm{comp}}
        \alpha_\Qarrow
        \alpha_\Rarrow
        \prod_{i | Q_i=R_i\neq I}
            \beta_{i,Q_i}^{-1}
        \prod_{i | Q_i\neq R_i}
            m_i
\end{align}

At some stage, I will need to go find out what the HF state looks like!

In [3]:
# dic = lih.pauli_rep.dic
# n = lih.pauli_rep.num_qubits

In [4]:
# dic_tf = dic.copy()
# dic_tf.pop('I'*n)

In [5]:
def is_influential_pauli_single_qubit(q, r):
    if q == r:
        return True
    elif q == 'I' and r == 'Z':
        return True
    elif q == 'Z' and r == 'I':
        return True
    else:
        return False

In [6]:
def is_influential_pair(Q, R, n):
    # do not check if Q or R are trace-free. Assume they are!
    for i in range(n):
        if is_influential_pauli_single_qubit(Q[i], R[i]) is False:
            return False
    # if I arrive here, then all qubits i have pauli Q_i R_i acceptable
    return True

In [7]:
def build_influential_pairs(dic_tf, n):
    # do not check if dic_tf has identity term. Assume it is not present!
    pairs = []
    for Q in dic_tf:
        for R in dic_tf:
            if is_influential_pair(Q, R, n) is True:
                pairs.append((Q,R))
    return pairs

In [8]:
def calculate_product_term(Q, R, bits_HF, β):
    # assume Q,R are already admissable!
    # assume β, m are of correct structure!
    pauli_to_index = {'X': 0, 'Y': 1, 'Z': 2}
    prod = 1.0
    for i in range(len(Q)):
        if Q[i] == R[i] and Q[i] != 'I':
            index = pauli_to_index[Q[i]]
            b = β[(len(Q)-1)-i][index]  # qiskit ordering
            if b == 0.0:
                # this cannot be allowed, as convergence in expectation won't work
                return float('inf')
            else:
                prod *= b**(-1)
        if Q[i] != R[i]: 
            # then Q[i], R[i] are of the form I,Z or Z,I
            bit = int(bits_HF[i])
            m = (-1)**bit
            prod *= m
    return prod

In [9]:
def objective(dic_tf, influential_pairs, bits_HF, β):
    tally = 0.0
    for Q,R in influential_pairs:
        alphaQ = dic_tf[Q]
        alphaR = dic_tf[R]
        prod = calculate_product_term(Q, R, bits_HF, β)
        tally += alphaQ * alphaR * prod
    return tally

In [10]:
from scipy.optimize import minimize, LinearConstraint

def x_to_beta(x):
    β = {}
    for i, (a, b, c) in enumerate(zip(x[0::3], x[1::3], x[2::3])):
        β[i] = [a, b, c]
    return β


def _lin_con_single(k, n):
    linear_constraint = [0]*(3*n)
    linear_constraint[k] = 1
    return linear_constraint


def _lin_con_triple(i, n):
    linear_constraint = [0]*(3*n)
    for var in [3*i, 3*i+1, 3*i+2]:
        linear_constraint[var] = 1
    return linear_constraint


def linear_constraint_matrix(n):
    # constraints to ensure \beta_{i,P} \ge 1 for all i,P
    mat1 = [_lin_con_single(k, n) for k in range(3*n)]
    # constraints to ensure \sum_P \beta_{i,P} = 1 for all i
    mat2 = [_lin_con_triple(i, n) for i in range(n)]
    return mat1+mat2


def lower_bounds(n):
    bounds_single = [0]*(3*n)
    bounds_triple = [1]*n
    return bounds_single+bounds_triple


def upper_bounds(n):
    return [1]*(4*n)


def constraints(n):
    A = linear_constraint_matrix(n)
    lb = lower_bounds(n)
    ub = upper_bounds(n)
    return LinearConstraint(A, lb, ub)


def find_optimal_beta(pauli_rep, bits_HF):
    n = pauli_rep.num_qubits
    dic_tf = pauli_rep.dic.copy()
    dic_tf.pop('I'*n)
    influential_pairs = build_influential_pairs(dic_tf, n)
    
    x0 = [1/3]*(3*n)

    def f(x):
        return objective(dic_tf, influential_pairs, bits_HF, x_to_beta(x))

    result = minimize(f, x0, method='trust-constr', constraints=[constraints(n)])
    β = x_to_beta(result['x'])
    return β

In [11]:
pr = lih.pauli_rep

## HOW DO I FIGURE OUT WHAT THE HARTREE-FOCK STATE LOOKS LIKE?

In [12]:
# this is definitely not the true HF state:
bits_HF = '0'*pr.num_qubits

In [13]:
%time β_optimal = find_optimal_beta(pr, bits_HF)
β_optimal

CPU times: user 6.41 s, sys: 549 ms, total: 6.95 s
Wall time: 2.35 s


{0: [0.22225752820348266, 0.20826986116466736, 0.5694726106318498],
 1: [0.2370558716560518, 0.23541476140504872, 0.5275293669388995],
 2: [0.22225751371451372, 0.2082698682278689, 0.5694726180576174],
 3: [0.23705587739261577, 0.23541475364211975, 0.5275293689652646]}

In [14]:
bits_HF = '1'*pr.num_qubits
%time β_optimal = find_optimal_beta(pr, bits_HF)
β_optimal

CPU times: user 1.2 s, sys: 114 ms, total: 1.32 s
Wall time: 442 ms


{0: [0.3378915109370397, 0.3388905396381586, 0.3232179494248017],
 1: [0.26257695444826173, 0.255524248903851, 0.4818987966478871],
 2: [0.33789151562933034, 0.3388905421975528, 0.3232179421731169],
 3: [0.2625769593743565, 0.25552425024835695, 0.4818987903772865]}